# AA228/CS238 Optional Final Project: Escape Roomba

This notebook demonstrates how to interact with the Roomba environment. We show how to:
1. Import the necessary packages
2. Define the sensor and construct the POMDP
3. Set up a particle filter
4. Define a policy
5. Simulate and render the environment
6. Evaluate the policy


In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"C:\\Users\\Joseph\\AA228FinalProject\\Project.toml"

In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

## Define sensor and construct POMDP
In the following cell, we first instantiate a sensor. There are two sensors implemented: a lidar sensor and a bump sensor. The lidar sensor receives a single noisy measurement of the distance to the nearest wall in the direction the Roomba is facing. The bump sensor indicates when contact has been made between any part of the Roomba and any wall.

Next, we instantiate the MDP, which defines the underlying simulation environment, assuming full observability. The MDP takes many arguments to specify details of the problem. Feel free to examine the underlying source code (```src/roomba_env.jl```) to gain an understanding for these arguments. One argument we must specify here is the ```config```. This argument, which can take values 1,2, or 3, specifies the room configuration, with each configuration corresponding to a different location for the goal and stairs.

Finally, we instantiate the POMDP. The POMDP takes as an argument the underlying MDP as well as the sensor, which it uses to define the observation model. 

In [3]:
sensor = Bumper() # or Bumper() for the bumper version of the environment
config = 3 # 1,2, or 3
#=
num_x_pts = 50
num_y_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts, num_y_pts, num_th_pts)

vlist = [0, 1, 5]
omlist = [-pi, -pi/2, -pi/4, 0, pi/4, pi/2, pi]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));
=#
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config));

### Setting up a Particle Filter

To solve a POMDP, we must specify a method for representing the belief state and updating it given an observation and an action. Here, we demonstrate how to instantiate a particle filter, which has been implemented in the package ```ParticleFilters.jl```.

First, we instantiate a resampler, which is responsible for updating the belief state given an observation. We have implemented a resampler for each of the lidar and bumper environments, the source code for which can be found in ```src/filtering.jl```. The first argument for both resamplers is the number of particles that represent the belief state. The lidar resampler takes a low-variance resampler as an additional argument, implemented for us in ```ParticleFilters.jl```, which is responsible for efficiently resampling a weighted set of particles. 

Next, we instantiate a ```SimpleParticleFilter```, which enables us to perform our belief updates, and is implemented for us in ```ParticleFilters.jl```, .

Finally, we pass this particle filter into a custom struct called a ```RoombaParticleFilter```, which takes two additional arguments. These arguments specify the noise in the velocity and turn-rate, used when propegating particles according to the action taken. These can be tuned depending on the type of sensor used.

In [4]:
num_particles = 2000
resampler = BumperResampler(num_particles)
# for the bumper environment
# resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

### Define a policy

Here we demonstrate how to define a naive policy that attempts navigate the Roomba to the goal. The heuristic policy we define here first spins around for 25 time-steps in order to perform localization, then follows a simple proprtional control law that navigates the robot in the direction of the goal state (note that this policy fails if there is a wall in the way).

First we create a struct that subtypes the Policy abstract type, defined in the package ```POMDPPolicies.jl```. Here, we can also define certain parameters, such as a variable tracking the current time-step.

Next, we define a function that can take in our policy and the belief state and return the desired action. We do this by defining a new ```POMDPs.action``` function that will work with our policy. 

In [14]:
# Define the policy to test
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

v_list = [0, 1, 5]
theta_list = [-pi/2, 0, pi / 2]
outfile = "hit_wall_action.dat"# = open("hit_wall_action.dat", "w")

# extract goal for heuristic controller
goal_xy = get_goal_xy(m)
println("goal_xy", goal_xy)

# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    p.ts += 1
    c = n_particles(b)
    s = particles(b)[c]
    println(s)
    #s = b[c]
    #s = mean(b)
    
    x = s[1]
    y = s[2]
    
    v = rand(v_list)
    theta = rand(theta_list)
    
    if wall_contact(m, [x,y])
        #println("Hit a wall")
        #return RoombaAct(5, -pi/4)    
        open(outfile, "a") do io
            @printf(io, "1,%f,%f\n", v, theta)        
        end
    else
        #println("didn't hit a wall")
        #return RoombaAct(5, 0)
        open(outfile, "a") do io
            @printf(io, "0,%f,%f\n", v, theta) 
        end
    end
    
    return RoombaAct(v, theta)
    #=
    #############
    # spin around to localize for the first 25 time-steps
    if p.ts < 25
        p.ts += 1
        return RoombaAct(0.,1.0) # all actions are of type RoombaAct(v,om)
    end
    p.ts += 1
    
    # after 25 time-steps, we follow a proportional controller to navigate
    # directly to the goal, using the mean belief state
    
    # compute mean belief of a subset of particles
    s = mean(b)
    
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = goal_xy
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    del_angle = wrap_to_pi(ang_to_goal - th)
    
    # apply proportional control to compute the turn-rate
    Kprop = 1.0
    om = Kprop * del_angle
    
    # always travel at some fixed velocity
    v = 5.0
    
    return RoombaAct(v, om)
    =#
end

goal_xy[15.0, 0.0]


### Simulation and rendering

Here, we will demonstrate how to seed the environment, run a simulation, and render the simulation. To render the simulation, we use the ```Gtk``` package. 

The simulation is carried out using the ```stepthrough``` function defined in the package ```POMDPSimulators.jl```. During a simulation, a window will open that renders the scene. It may be hidden behind other windows on your desktop.

In [15]:
# first seed the environment
Random.seed!(2)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=15))
    @guarded draw(c) do widget
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.01) # to slow down the simulation
end

[-12.767, -1.53567, -2.71682, 0.0]
[1.52852, -2.67236, 0.491672, 0.0]
[-21.3821, -4.94895, 1.53009, 0.0]
[6.69504, -1.76502, 1.33261, 0.0]
[-10.2254, 3.65518, 2.67564, 0.0]
[-10.9937, -3.07662, -1.77417, 0.0]
[-19.3056, -8.22158, -2.0011, 0.0]
[-22.8827, -7.2735, 1.37257, 0.0]
[4.92549, -0.274552, 2.24182, 0.0]
[-1.56402, -1.19154, 1.9655, 0.0]
[-21.3945, -5.64133, 0.918129, 0.0]
[-18.3677, -6.34047, 0.0920829, 0.0]
[-22.2106, -9.33896, -2.14379, 0.0]
[-4.54644, -2.33318, -0.463476, 0.0]
[-19.7336, -4.582, -3.12021, 0.0]


### Specifying initial states and beliefs
If, for debugging purposes, you would like to hard-code a starting location or initial belief state for the roomba, you can do so by taking the following steps.

First, we define the initial state using the following line of code:
```
is = RoombaState(x,y,th,0.)
```
Where ```x``` and ```y``` are the x,y coordinates of the starting location and ```th``` is the heading in radians. The last entry, ```0.```, respresents whether the state is terminal, and should remain unchanged.

If you would like to initialize the Roomba's belief as perfectly localized, you can do so with the following line of code:
```
b0 = Deterministic(is)
```
If you would like to initialize the standard unlocalized belief, use these lines:
```
dist = initialstate_distribution(m)
b0 = initialize_belief(belief_updater, dist)
```
Finally, we call the ```stepthrough``` function using the initial state and belief as follows:
```
stepthrough(m,planner,belief_updater,b0,is,max_steps=300)
```

### Discretizing the state space
Certain POMDP solution techniques require discretizing the state space. Should we need to do so, we first define the state space by specifying the number of points to discretize the range of possible x, y, and $\theta$ values, and then calling the DiscreteRoombaStateSpace constructor.
```
num_x_pts = ... # e.g. 50
num_y_pts = ... # e.g. 50
num_th_pts = ... # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)
```

Next, we pass in the state space as an argument when constructing the POMDP.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace))
```

### Discretizing the action space
Certain POMDP solution techniques require discretizing the action space. Should we need to do so, we first define the action space as follows:
```
vlist = [...]
omlist = [...]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))
```
Where ```vlist``` is an array of possible values for the velocity (e.g. ```[0,1,10]```) and ```omlist``` is an array of possible turn-rates (e.g. ```[-1,0,1]```).

Next, we pass in the action space as an argument when constructing the POMDP.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, aspace=aspace))
```

### Discretizing the Lidar observation space
Certain POMDP solution techniques require discretizing the observation space. The Bumper sensor has a discrete observation space by default, while the Lidar sensor is continuous by default. The observations can take values in the range $[0,\infty)$. Should we need to do discretize the Lidar observation space, we first define cut-points $x_{1:K}$ such that all observations between $-\infty$ and $x_1$ are considered the discrete observation 1, all observations between $x_1$ and $x_2$ are considered discrete observation 2, and so on. All observations between $x_K$ and $\infty$ are considered discrete observation $K+1$. We instantiate the discretized sensor as follows:
```
cut_points = [x_1, x_2, ..., x_K]
sensor = DiscreteLidar(cut_points)
```
Next, we pass in the sensor as an argument when constructing the POMDP as usual.
```
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config))
```






# Evaluation 

Here, we demonstate a simple evaluation of the policy's performance for a few random seeds. This is meant to serve only as an example, and we encourage you to develop your own evaluation metrics.

We intialize the robot using five different random seeds, and simulate its performance for 100 time-steps. We then sum the rewards experienced during its interaction with the environment and track this total reward for the five trials.
Finally, we report the mean and standard error for the total reward. The standard error is the standard deviation of a sample set divided by the square root of the number of samples, and represents the uncertainty in the estimate of the mean value.

In [16]:
using Statistics

total_rewards = []

for exp = 1:5
    println(string(exp))
    
    Random.seed!(exp)
    
    p = ToEnd(0)
    traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=100)])
    
    push!(total_rewards, traj_rewards)
end

@printf("Mean Total Reward: %.3f, StdErr Total Reward: %.3f", mean(total_rewards), std(total_rewards)/sqrt(5))

1
[-16.5499, -17.9516, 0.903185, 0.0]
[11.9556, 1.38436, 2.67151, 0.0]
[-17.7701, -15.2646, -0.399924, 0.0]
[-14.4361, 0.792948, 1.51057, 0.0]
[-15.5, -11.2187, -0.977323, 0.0]
[-15.5, -18.6095, 0.679107, 0.0]
[-15.5, -15.5184, -0.346133, 0.0]
[-17.5165, 4.5, 2.57054, 0.0]
[-15.5, -16.1622, 1.23403, 0.0]
[-15.5, -16.1622, 0.734028, 0.0]
[-15.5, -16.1622, 0.829693, 0.0]
[9.74061, -4.5, -1.0997, 0.0]
[-19.358, -19.5, -0.0284487, 0.0]
[-11.1433, 4.5, 2.63645, 0.0]
[-11.1452, 4.5, 2.97131, 0.0]
[7.30663, -4.5, -0.388033, 0.0]
[4.50778, -4.5, -0.533881, 0.0]
[4.50778, -4.5, -1.03388, 0.0]
[4.50778, -4.5, -1.53388, 0.0]
[4.50778, -4.5, -1.03388, 0.0]
[4.50778, -4.5, -0.533881, 0.0]
[-13.0407, -4.5, -0.439696, 0.0]
[-24.3134, -8.51026, -1.3131, 0.0]
[0.796662, 4.43931, -3.0666, 0.0]
[-12.7007, 3.92259, -2.7772, 0.0]
[-19.8071, 3.57693, -3.09052, 0.0]
[-22.1041, 4.14226, 3.0512, 0.0]
[0.604477, -4.5, -0.556552, 0.0]
[-15.5, -9.33719, 0.770774, 0.0]
[-15.5, -9.33719, 0.270774, 0.0]
[-15.5, -9.3

[1.15384, -4.5, -0.559339, 0.0]
[-6.07325, 4.5, 2.83178, 0.0]
[-3.4507, 4.41822, -3.04573, 0.0]
[-0.690057, 4.33073, -2.66823, 0.0]
[-7.61125, 4.45484, 2.92344, 0.0]
[-1.03624, 4.49292, 3.07182, 0.0]
4
[-1.41532, 2.57554, -1.16385, 0.0]
[-3.59842, 3.90765, 2.87482, 0.0]
[-17.9571, 0.687531, 1.6645, 0.0]
[-21.1252, 3.63129, 2.35965, 0.0]
[6.69767, 2.52892, 2.59781, 0.0]
[-4.853, -3.62551, 0.0779751, 0.0]
[-22.3022, -16.9332, 1.5632, 0.0]
[-15.9801, -6.34463, 2.16697, 0.0]
[-14.343, -1.09737, 1.73514, 0.0]
[-20.5432, -0.850102, -1.26328, 0.0]
[-16.7686, -11.5945, -2.03472, 0.0]
[-17.5304, -12.339, 1.08288, 0.0]
[-22.1127, -19.5, -1.65955, 0.0]
[-5.20078, 4.5, 3.02548, 0.0]
[7.00033, 4.5, 2.4057, 0.0]
[-7.02051, 4.5, 0.785182, 0.0]
[-20.6222, 4.5, 0.884407, 0.0]
[1.03527, -4.5, -0.804417, 0.0]
[-12.8265, -4.5, -1.5497, 0.0]
[13.2968, -4.5, -1.64715, 0.0]
[13.2968, -4.5, -1.14715, 0.0]
[13.2968, -4.5, -0.730167, 0.0]
[-7.44148, 3.9013, -2.91815, 0.0]
[-24.5, -15.5326, -2.0505, 0.0]
[-24.5,